# SQL Language 


#### SQL Queries, Joins, Aggregation and Date functions 


in Python Jupiter notebook environment 


In [1]:
#imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3  

import warnings
warnings.filterwarnings('ignore')

### SQL Queries

- Create a database

- Create a table in it

- Insert Data into table 

- Retrieve (View) table 

- Extract records that fulfill a specified criterion(a)

- Modify (Update) records

- Modify (Alter) columns


### Create a connection object that represents the database

In [4]:
conn = sqlite3.connect('DB1.db')

print("Opened/Created database successfully")

Opened/Created database successfully


###  Create a table in the database 

using SQL CREATE TABLE statement


In [5]:
conn.execute('''CREATE TABLE table1
         (ID INT PRIMARY KEY     NOT NULL,
         NAME           TEXT    NOT NULL,
         LOCATION       CHAR(50),
         EMAIL         CHAR(50));''')

print("Table created successfully")

Table created successfully


### Insert data into the table 

using SQL INSERT INTO statement

In [6]:
conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (1, 'Mary', 'LA', 'mary@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (2, 'Alex', 'Miami', 'alex@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (3, 'Kyra', 'NYC', 'kyra@123.com' )");

conn.execute("INSERT INTO table1 (ID,NAME,LOCATION,EMAIL) \
      VALUES (4, 'Nancy', 'SF', 'nancy@123.com' )");

conn.commit()
print("Records inserted successfully")

Records inserted successfully


### View table with Pandas 

using SQL SELECT * FROM statement

In [7]:
t1 = pd.read_sql("""SELECT * FROM table1;""", conn)
t1

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,mary@123.com
1,2,Alex,Miami,alex@123.com
2,3,Kyra,NYC,kyra@123.com
3,4,Nancy,SF,nancy@123.com


### SQL WHERE clause

filtering based on a condition(s)

In [8]:
#Get the NAME where LOCATION is NYC:
pd.read_sql("""SELECT NAME FROM table1
                    WHERE LOCATION = 'NYC';""", conn)

,NAME
0,Kyra


In [9]:
#Get the NAME AND EMAIL where LOCATION is SF or NYC:
pd.read_sql("""SELECT NAME, EMAIL FROM table1
                            WHERE LOCATION = 'NYC' OR LOCATION ='SF';""", conn)

,NAME,EMAIL
0,Kyra,kyra@123.com
1,Nancy,nancy@123.com


### SQL UPDATE Query

to modify row(s)/record(s) in table - use WHERE clause to specifies which record(s) should be updated


In [10]:
#Change the lOCATION for Alex from Miami to Montreal using the following query:
query = """UPDATE table1 SET LOCATION = 'Montreal' WHERE NAME = 'Alex';"""

#execute the query
conn.execute(query)
conn.commit()

In [11]:
#Check Alex record to see if it is updated 
pd.read_sql("""SELECT LOCATION FROM table1
                            WHERE NAME = 'Alex';""", conn)

,LOCATION
0,Montreal


In [12]:
#View updated table
pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,mary@123.com
1,2,Alex,Montreal,alex@123.com
2,3,Kyra,NYC,kyra@123.com
3,4,Nancy,SF,nancy@123.com


### SQL DELETE Query 

to delete row(s) from a table - use WHERE clause to specifies which record(s) should be deleted

In [13]:
#Delete the last row use the following query:
query = """DELETE FROM table1 WHERE ID = 4;"""

#Execute the query
conn.execute(query)
conn.commit()

In [14]:
#View table to check if 4th row was deleted
pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL
0,1,Mary,LA,mary@123.com
1,2,Alex,Montreal,alex@123.com
2,3,Kyra,NYC,kyra@123.com


### SQL ALTER TABLE Query

to change the structure of a table:

add/delete/rename columns or change columns or constraints 


In [15]:
#Add a new column, age of integer type, to the table using the following query:
query = """ALTER TABLE table1 ADD age int;"""

#Execute the query
conn.execute(query)
conn.commit()

In [16]:
#Insert age values updating the table using the following queries:

conn.execute("UPDATE table1 SET age = 28 where Name = 'Mary'");
conn.execute("UPDATE table1 SET age = 31 where Name = 'Alex'");
conn.execute("UPDATE table1 SET age = 33 where Name = 'Kyra'");

In [17]:
#View table to check if the new age column was added
pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL,age
0,1,Mary,LA,mary@123.com,28
1,2,Alex,Montreal,alex@123.com,31
2,3,Kyra,NYC,kyra@123.com,33


So far, I used above SQL queries on 1 table. However, in practice data is shared accross multiple tables. 


Using SQL Joins one can use SQL queries on multiple tables: 



### SQL Joins


- inner join

- left join

- right join

- full outer join

- cross join



In [18]:
# First, I create another table in my database (using SQL CREATE TABLE statement)

conn.execute('''CREATE TABLE table2
         (ID INT PRIMARY KEY     NOT NULL,
         SALARY REAL);''')

print("Table created successfully")

Table created successfully


In [19]:
#Insert some data into a second table (using SQL INSERT INTO statement)

conn.execute("INSERT INTO table2 values (1, 100000)");

conn.execute("INSERT INTO table2 values (2, 125000)");


In [20]:
#View table2

t2= pd.read_sql("""SELECT * FROM table2;""", conn)
t2

,ID,SALARY
0,1,100000.0
1,2,125000.0


In [21]:
#View table1
pd.read_sql("""SELECT * FROM table1;""", conn)

,ID,NAME,LOCATION,EMAIL,age
0,1,Mary,LA,mary@123.com,28
1,2,Alex,Montreal,alex@123.com,31
2,3,Kyra,NYC,kyra@123.com,33


### SQL INNER JOIN the two tables on a common column

get common rows from the two tables

In [22]:
#get name, location and salary from table1 & table2 using the following SQL inner join on ID:
conn.execute("SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A INNER JOIN table2 B ON A.ID = B.ID").fetchall()

[('Mary', 'LA', 100000.0), ('Alex', 'Montreal', 125000.0)]

In [23]:
#Alternatively, using Pandas
pd.read_sql("SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A INNER JOIN table2 B ON A.ID = B.ID", conn)

,NAME,LOCATION,SALARY
0,Mary,LA,100000.0
1,Alex,Montreal,125000.0


### SQL LEFT JOIN on a common column

common rows + all the rows from table1 replacing missing values with NaN

In [24]:
#Get the name, location and salary from table1 & table2 using the following SQL left join on ID:

pd.read_sql("SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A LEFT JOIN table2 B ON A.ID = B.ID", conn)


,NAME,LOCATION,SALARY
0,Mary,LA,100000.0
1,Alex,Montreal,125000.0
2,Kyra,NYC,NaN


Although this Python environment supports only left & inner joins, other joins queries may be useful:

### SQL RIGHT JOIN  on a common column

common rows + all the rows from table2 replacing missing values with NaN

In [26]:
query ="SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A RIGHT JOIN table2 B ON A.ID = B.ID";

### SQL FULL OUTER JOIN on a common column

all records from table1 and table2 replacing missing values with NaN

In [27]:
query = "SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A CROSS JOIN table2 B ON A.ID = B.ID";

### SQL CROSS JOIN 

all records from table1 combined with all rows from table2  

In [28]:
query = "SELECT A.NAME, A.LOCATION, B.SALARY FROM table1 A CROSS JOIN table2 B ON A.ID = B.ID";

### SQL Aggregation & Date Functions

In [29]:
#view table2
pd.read_sql("""SELECT * FROM table2;""", conn)

,ID,SALARY
0,1,100000.0
1,2,125000.0


In [30]:
#Average function
pd.read_sql("""SELECT AVG(SALARY) from table2;""", conn)

,AVG(SALARY)
0,112500.0


In [31]:
#count function
pd.read_sql("SELECT COUNT(SALARY) FROM table2;", conn)

,COUNT(SALARY)
0,2


In [32]:
#sum
pd.read_sql("SELECT SUM(SALARY) FROM table2;", conn)

,SUM(SALARY)
0,225000.0


In [33]:
#MAX 
pd.read_sql("SELECT MAX(SALARY) FROM table2;", conn)

,MAX(SALARY)
0,125000.0


In [34]:
#MIN
pd.read_sql("SELECT MIN(SALARY) FROM table2;", conn)

,MIN(SALARY)
0,100000.0


In [35]:
#DATE 
pd.read_sql("SELECT DATE();", conn)

,DATE()
0,2021-04-27
